In [1]:
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator, EffectSimulatorCollection
from shared.analyzers import AnalyzerCollection, WLSAnalyzer, IPWAnalyzer, SIPWAnalyzer, MetaAnalyzer
from shared.result_processors import Results
from tqdm import trange
import matplotlib.pyplot as plt
import copy

In [2]:
N = 50
T = 100
alpha = 0.05
n_reps = 100
weight_type = "ATE"

# Effect simulators
effect_simulators = EffectSimulatorCollection()
effect_simulators.add_effect_simulator(ConstantEffectSimulator("No Effect", N, 0., 0., 0.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Shared Effect", N, 1., 1., 0.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Distinct Effects", N, 1., 1., 0.3))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Moderated Effects", N, 1., 1., 0.3, 1., 0.2))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Noisy Moderated Effects", N, 1., 1., 0.3, 1., 0.2, epsilon_sd=1.))

# Simulate effects once and fix
for effect_simulator in effect_simulators:
    effect_simulator.sample_params()

# Analyzers
analyzers = AnalyzerCollection()
wls_analyzer = WLSAnalyzer("WLS", alpha)
analyzers.add_analyzer(wls_analyzer)
analyzers.add_analyzer(MetaAnalyzer(wls_analyzer, "Meta-WLS", alpha))

ipw_analyzer = IPWAnalyzer("IPW", alpha)
analyzers.add_analyzer(IPWAnalyzer("IPW", alpha))
analyzers.add_analyzer(MetaAnalyzer(ipw_analyzer, "Meta-IPW", alpha))

sipw_analyzer = SIPWAnalyzer("SIPW", alpha)
analyzers.add_analyzer(sipw_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_analyzer, "Meta-SIPW", alpha))

ipw_dr_analyzer = IPWAnalyzer("IPW-DR", alpha, dr=True)
analyzers.add_analyzer(ipw_dr_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_dr_analyzer, "Meta-IPW-DR", alpha))

sipw_dr_analyzer = SIPWAnalyzer("SIPW-DR", alpha, dr=True)
analyzers.add_analyzer(sipw_dr_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_dr_analyzer, "Meta-SIPW-DR", alpha))

# Simulation loop
data_simulator = DataSimulator(N, T).simulate_history()
results = Results(data_simulator, effect_simulators, analyzers, n_reps=n_reps)
for rep_idx in trange(n_reps):
    for effect_simulator in effect_simulators:
        data_simulator = data_simulator.simulate_a()
        data_simulator = data_simulator.simulate_effects(effect_simulator)
        for analyzer in analyzers:          
            # Inferences for theta
            theta_inferences = analyzer.get_theta_inferences(data_simulator)
            results = results.add_theta_inferences(theta_inferences, data_simulator.theta.copy(), rep_idx, effect_simulator.i, analyzer.i)
            
            # Inferences for user estimates
            user_inferences = analyzer.get_user_inferences(data_simulator)
            results = results.add_user_inferences(user_inferences, data_simulator.user_effects.copy(), rep_idx, effect_simulator.i, analyzer.i)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 16.14it/s]


In [3]:
digits = 3
results = results.process(digits=digits)
results.user_results_df

WLS  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     No Effect                0.178 (0.003)   
                         Shared Effect            0.180 (0.003)   
                         Distinct Effects         0.174 (0.003)   
                         Moderated Effects        0.481 (0.003)   
                         Noisy Moderated Effects  0.458 (0.004)   
            Avg Sq. Bias No Effect                        0.038   
                         Shared Effect                    0.038   
                         Distinct Effects                 0.037   
                         Moderated Effects                0.253   
                         Noisy Moderated Effects          0.232   
            MSE          No Effect                0.083 (0.001)   
                         Shared Effect            0.081 (0.001)   
                         Distinct Effects         0.081 (0.001)   
                         Moderated Effects        0.321 (0.004)   
                         Noisy Moderated Effects  0.307 (0.004)   
            Coverage     No Effect                0.853 (0.006)   
                         Shared Effect            0.860 (0.005)   
                         Distinct Effects         0.859 (0.005)   
                         Moderated Effects        0.635 (0.006)   
                         Noisy Moderated Effects  0.710 (0.005)   
            CI Length    No Effect                0.836 (0.000)   
                         Shared Effect            0.836 (0.000)   
                         Distinct Effects         0.836 (0.000)   
                         Moderated Effects        1.137 (0.001)   
                         Noisy Moderated Effects  1.238 (0.001)   

                                                       Meta-WLS  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     No Effect                0.177 (0.003)   
                         Shared Effect            0.178 (0.003)   
                         Distinct Effects         0.174 (0.003)   
                         Moderated Effects        0.467 (0.003)   
                         Noisy Moderated Effects  0.444 (0.004)   
            Avg Sq. Bias No Effect                        0.034   
                         Shared Effect                    0.035   
                         Distinct Effects                 0.042   
                         Moderated Effects                0.240   
                         Noisy Moderated Effects          0.226   
            MSE          No Effect                0.056 (0.001)   
                         Shared Effect            0.055 (0.001)   
                         Distinct Effects         0.063 (0.001)   
                         Moderated Effects        0.277 (0.003)   
                         Noisy Moderated Effects  0.255 (0.004)   
            Coverage     No Effect                0.861 (0.006)   
                         Shared Effect            0.865 (0.006)   
                         Distinct Effects         0.834 (0.006)   
                         Moderated Effects        0.524 (0.007)   
                         Noisy Moderated Effects  0.520 (0.009)   
            CI Length    No Effect                0.691 (0.000)   
                         Shared Effect            0.691 (0.000)   
                         Distinct Effects         0.692 (0.000)   
                         Moderated Effects        0.959 (0.002)   
                         Noisy Moderated Effects  0.932 (0.005)   

                                                             IPW  \
Coefficient Metric       Effect Simulator                          
Intercept   Avg Bias     No Effect                 0.001 (0.006)   
                         Shared Effect             0.009 (0.005)   
                         Distinct Effects         -0.012 (0.006)   
                         Moderated Effects         0.007 (0.006)   
                         Noisy M